In [ ]:
import numpy as np
import librosa


# 
y, sr = librosa.load('out.wav', sr=44100)

print(y.shape, sr)
print(y.shape[0]/44100)
f0, voiced_flag, voiced_probs = librosa.pyin(y,
                                            sr=16000,
                                            fmin=librosa.note_to_hz('C2'),
                                            fmax=librosa.note_to_hz('C7'),
                                            fill_na=0.0)

print(voiced_flag.shape)

f02, voiced_flag2, voiced_probs2 = librosa.pyin(y,
                                            sr=44100,
                                            fmin=librosa.note_to_hz('C2'),
                                            fmax=librosa.note_to_hz('C7'),
                                            fill_na=0.0)

print(f02)
# print(features.shape)
# print(features.shape[1]/44100)



In [ ]:
import webrtcvad



def get_vad(y, sr, mode=2):
    vad = webrtcvad.Vad()
    vad.set_mode(mode)

    sample_rate = 16000
    # downsampling from 44100 to 16000 and framing
    y_16k = librosa.resample(y, orig_sr=sr, target_sr=sample_rate)
    # transfer to bytes
    y_16k = np.int16(y_16k * 32768).tobytes()
    # calculate bytes of 10ms in 16000Hz
    step = int(sample_rate * 0.01) * 2

    # assert error if len(y_16k) < step
    assert len(y_16k) >= step

    # loop the bytes in 10ms frames
    frames = []
    for i in range(0, len(y_16k) - step, step):
        frames.append(y_16k[i:i+step])

    vadres = []
    for i, frame in enumerate(frames):
        vadres.append(vad.is_speech(frame, sample_rate))
    return vadres

vadres = get_vad(y, sr, mode=3)
print(len(vadres))
print(vadres)
start = np.argmax(vadres) / 100.0
end = (len(vadres) - np.argmax(np.flip(vadres))) / 100.0
print(start, end)



In [1]:
from silero_vad import load_silero_vad, read_audio, get_speech_timestamps

fn = "/data/storage025/Turntaking/PD_batch3_raw_single_channel_normalized/EarlyLate/subj-2236_14_E_beweegt_strand.wav_1.wav"
model = load_silero_vad()
wav = read_audio(fn) # backend (sox, soundfile, or ffmpeg) required!
print(wav.shape)
speech_timestamps = get_speech_timestamps(wav, model)
print(speech_timestamps)
print(speech_timestamps[0]['start']/16000, speech_timestamps[0]['end']/16000)


torch.Size([41239])
[]


IndexError: list index out of range